In [2]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import tensorflow as tf
sns.set(style="whitegrid")
import torch
import torchvision.models as models
import joblib
np.random.seed(203)
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

In [32]:
data = pd.read_csv('C:/Users/user/Desktop/최적입지_변수_총합_가중치(상관)_최종.csv', encoding = 'cp949')

In [33]:
data

,id,bus,metro,park,high,uni,age,pop,mart,Class
0,28208,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0
1,28207,0.000000,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,0
2,28055,0.000000,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,0
3,28054,0.062653,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,0
4,28053,0.125306,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,1
...,...,...,...,...,...,...,...,...,...,...
15672,233,0.125306,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0
15673,232,0.125306,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0
15674,82,0.031326,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0
15675,81,0.093979,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0


In [34]:
data.drop(columns = ['id'], inplace =True)

In [35]:
data

,bus,metro,park,high,uni,age,pop,mart,Class
0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0
1,0.000000,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,0
2,0.000000,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,0
3,0.062653,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,0
4,0.125306,0.0,0.0,0.0,0.0,0.008674,-0.002736,0.0,1
...,...,...,...,...,...,...,...,...,...
15672,0.125306,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0
15673,0.125306,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0
15674,0.031326,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0
15675,0.093979,0.0,0.0,0.0,0.0,0.026023,-0.003648,0.0,0


In [36]:
vc = data['Class'].value_counts().to_frame().reset_index()
vc['percent'] = vc["Class"].apply(lambda x : round(100*float(x) / len(data), 2))
vc = vc.rename(columns = {"index" : "Target", "Class" : "Count"})
vc

,Target,Count,percent
0,0,10815,68.99
1,1,4862,31.01


In [37]:
non_fraud = data[data['Class'] == 0].sample(1000)
fraud = data[data['Class'] == 1]

df = non_fraud.append(fraud).sample(frac=1).reset_index(drop=True)
X = df.drop(['Class'], axis = 1).values
Y = df["Class"].values

C:\Users\user\AppData\Local\Temp\ipykernel_6284\3091554375.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = non_fraud.append(fraud).sample(frac=1).reset_index(drop=True)


In [38]:
## input layer 
input_layer = Input(shape=(X.shape[1],))

## encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

## decoding part
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

## output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

In [39]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

In [40]:
x = data.drop(["Class"], axis=1)
y = data["Class"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]

In [46]:
autoencoder.fit(x_norm[0:2000], x_norm[0:2000],  epochs = 200, shuffle = True, validation_split = 0.25);

Epoch 1/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0993 - val_loss: 0.1031
Epoch 2/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0974 - val_loss: 0.1014
Epoch 3/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0956 - val_loss: 0.0998
Epoch 4/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0939 - val_loss: 0.0983
Epoch 5/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0923 - val_loss: 0.0968
Epoch 6/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0907 - val_loss: 0.0954
Epoch 7/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0892 - val_loss: 0.0940
Epoch 8/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0878 - val_loss: 0.0927
Epoch 9/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0864 - val_loss: 0.0914
Epoch 10/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0851 - val_loss: 0.0903
Epoch 11/

47/47 [==============================] - 0s 2ms/step - loss: 0.0368 - val_loss: 0.0421
Epoch 84/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0366 - val_loss: 0.0419
Epoch 85/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0364 - val_loss: 0.0417
Epoch 86/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0363 - val_loss: 0.0415
Epoch 87/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0361 - val_loss: 0.0413
Epoch 88/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0359 - val_loss: 0.0411
Epoch 89/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0358 - val_loss: 0.0409
Epoch 90/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0356 - val_loss: 0.0407
Epoch 91/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0354 - val_loss: 0.0405
Epoch 92/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0353 - val_loss: 0.0403
Epoch 93/200


In [47]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

In [48]:
norm_hid_rep = hidden_representation.predict(x_norm[:950])
fraud_hid_rep = hidden_representation.predict(x_fraud)

152/152 [==============================] - 0s 1ms/step


In [49]:
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

In [50]:
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))


Classification Report: 
              precision    recall  f1-score   support

         0.0       0.84      0.46      0.59       244
         1.0       0.90      0.98      0.94      1209

    accuracy                           0.89      1453
   macro avg       0.87      0.72      0.77      1453
weighted avg       0.89      0.89      0.88      1453


Accuracy Score:  0.8947006194081212


In [22]:
model = 